In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv('attrition_data.csv')
print('Dataframe shape: ', df.shape)

Dataframe shape:  (9612, 27)


In [3]:
df.columns

Index(['EMP_ID', 'ANNUAL_RATE', 'HRLY_RATE', 'JOBCODE', 'ETHNICITY', 'SEX',
       'MARITAL_STATUS', 'JOB_SATISFACTION', 'AGE', 'NUMBER_OF_TEAM_CHANGED',
       'REFERRAL_SOURCE', 'HIRE_MONTH', 'REHIRE', 'TERMINATION_YEAR',
       'IS_FIRST_JOB', 'TRAVELLED_REQUIRED', 'PERFORMANCE_RATING',
       'DISABLED_EMP', 'DISABLED_VET', 'EDUCATION_LEVEL', 'STATUS',
       'JOB_GROUP', 'PREVYR_1', 'PREVYR_2', 'PREVYR_3', 'PREVYR_4',
       'PREVYR_5'],
      dtype='object')

In [4]:
df.isnull().sum()

EMP_ID                       0
ANNUAL_RATE                  0
HRLY_RATE                    0
JOBCODE                      0
ETHNICITY                    0
SEX                          0
MARITAL_STATUS               0
JOB_SATISFACTION             0
AGE                          0
NUMBER_OF_TEAM_CHANGED       0
REFERRAL_SOURCE            445
HIRE_MONTH                   0
REHIRE                       0
TERMINATION_YEAR          5394
IS_FIRST_JOB                 0
TRAVELLED_REQUIRED           0
PERFORMANCE_RATING           0
DISABLED_EMP                 0
DISABLED_VET                 0
EDUCATION_LEVEL              0
STATUS                       0
JOB_GROUP                    0
PREVYR_1                     0
PREVYR_2                     0
PREVYR_3                     0
PREVYR_4                     0
PREVYR_5                     0
dtype: int64

In [5]:
df.drop(['EMP_ID', 'JOBCODE', 'TERMINATION_YEAR'], axis=1, inplace=True)
df.drop(df.iloc[:, -5:], axis=1, inplace=True)

In [6]:
df['REFERRAL_SOURCE'].fillna(df['REFERRAL_SOURCE'].mode()[0], inplace=True)

In [7]:
df.head()

ANNUAL_RATE  HRLY_RATE ETHNICITY SEX MARITAL_STATUS  JOB_SATISFACTION  AGE  \
0        33615         22     BLACK   F       Divorced                 4   35   
1        70675         40     ASIAN   M         Single                 3   18   
2        34320         23     WHITE   F         Single                 5   18   
3       103199         59     ASIAN   F         Single                 2   50   
4       141801         71     ASIAN   F         Single                 4   34   

  NUMBER_OF_TEAM_CHANGED         REFERRAL_SOURCE HIRE_MONTH  REHIRE  \
0                     3+         Client Referral       June   False   
1                     3+      Executive Referral       June   False   
2                      2  Former Employee/Intern   February   False   
3                      0         Direct Sourcing    October    True   
4                      3                 Inroads        May   False   

  IS_FIRST_JOB TRAVELLED_REQUIRED  PERFORMANCE_RATING DISABLED_EMP  \
0            N                  N                   4            N   
1            Y                  N                   3            N   
2            Y                  N                   3            N   
3            N                  Y                   2            N   
4            N                  N                   4            N   

  DISABLED_VET EDUCATION_LEVEL STATUS                       JOB_GROUP  
0            N         LEVEL 2      T  Plant & Facilities Maintenance  
1            N         LEVEL 1      A                   Customer Care  
2            N         LEVEL 1      A                   Customer Care  
3            N         LEVEL 1      T                         Finance  
4            N         LEVEL 1      A              Marketing - Direct

In [8]:
le = LabelEncoder()
df['NUMBER_OF_TEAM_CHANGED'] = le.fit_transform(df['NUMBER_OF_TEAM_CHANGED'])
df['REHIRE'] = le.fit_transform(df['REHIRE'])
df['IS_FIRST_JOB'] = le.fit_transform(df['IS_FIRST_JOB'])
df['TRAVELLED_REQUIRED'] = le.fit_transform(df['TRAVELLED_REQUIRED'])
df['DISABLED_EMP'] = le.fit_transform(df['DISABLED_EMP'])
df['DISABLED_VET'] = le.fit_transform(df['DISABLED_VET'])
df['EDUCATION_LEVEL'] = le.fit_transform(df['EDUCATION_LEVEL'])
df['STATUS'] = le.fit_transform(df['STATUS'])

In [9]:
df

ANNUAL_RATE  HRLY_RATE ETHNICITY SEX MARITAL_STATUS  JOB_SATISFACTION  \
0           33615         22     BLACK   F       Divorced                 4   
1           70675         40     ASIAN   M         Single                 3   
2           34320         23     WHITE   F         Single                 5   
3          103199         59     ASIAN   F         Single                 2   
4          141801         71     ASIAN   F         Single                 4   
...           ...        ...       ...  ..            ...               ...   
9607        34636         23     ASIAN   F         Single                 1   
9608        34962         27     WHITE   F         Single                 5   
9609        39286         27     WHITE   F         Single                 3   
9610        39145         27     WHITE   M        Married                 4   
9611        90726         50     WHITE   F         Single                 1   

      AGE  NUMBER_OF_TEAM_CHANGED                REFERRAL_SOURCE HIRE_MONTH  \
0      35                       4                Client Referral       June   
1      18                       4             Executive Referral       June   
2      18                       2         Former Employee/Intern   February   
3      50                       0                Direct Sourcing    October   
4      34                       3                        Inroads        May   
...   ...                     ...                            ...        ...   
9607   43                       0                Staffing Agency       July   
9608   28                       2  Job Boards/Online Advertising   November   
9609   31                       2             Executive Referral   February   
9610   32                       4                     Open House   February   
9611   46                       0                    Job Posting   November   

      REHIRE  IS_FIRST_JOB  TRAVELLED_REQUIRED  PERFORMANCE_RATING  \
0          0             0                   0                   4   
1          0             1                   0                   3   
2          0             1                   0                   3   
3          1             0                   1                   2   
4          0             0                   0                   4   
...      ...           ...                 ...                 ...   
9607       0             0                   0                   1   
9608       0             0                   0                   1   
9609       0             0                   0                   1   
9610       0             0                   0                   4   
9611       0             0                   0                   1   

      DISABLED_EMP  DISABLED_VET  EDUCATION_LEVEL  STATUS  \
0                0             0                1       1   
1                0             0                0       0   
2                0             0                0       0   
3                0             0                0       1   
4                0             0                0       0   
...            ...           ...              ...     ...   
9607             0             1                0       0   
9608             0             0                2       0   
9609             0             0                2       0   
9610             0             0                1       0   
9611             0             0                3       0   

                           JOB_GROUP  
0     Plant & Facilities Maintenance  
1                      Customer Care  
2                      Customer Care  
3                            Finance  
4                 Marketing - Direct  
...                              ...  
9607                  Physical Flows  
9608                  Physical Flows  
9609         Production & Operations  
9610         Production & Operations  
9611                  Physical Flows  

[9612 rows x 19 columns]

In [10]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(df.corr(), annot = True, ax=ax)

<AxesSubplot:>

In [11]:
df.drop(['HRLY_RATE'], axis=1, inplace=True)

In [12]:
df['HIRE_MONTH'] = df['HIRE_MONTH'].astype('category')
df['JOB_GROUP'] = df['JOB_GROUP'].astype('category')
df['REFERRAL_SOURCE'] = df['REFERRAL_SOURCE'].astype('category')
df['ETHNICITY'] = df['ETHNICITY'].astype('category')
df['SEX'] = df['SEX'].astype('category')
df['MARITAL_STATUS'] = df['MARITAL_STATUS'].astype('category')
df = pd.get_dummies(df, columns=['HIRE_MONTH', 'JOB_GROUP', 'REFERRAL_SOURCE', 'SEX', 'MARITAL_STATUS', 'ETHNICITY'])

In [13]:
df

ANNUAL_RATE  JOB_SATISFACTION  AGE  NUMBER_OF_TEAM_CHANGED  REHIRE  \
0           33615                 4   35                       4       0   
1           70675                 3   18                       4       0   
2           34320                 5   18                       2       0   
3          103199                 2   50                       0       1   
4          141801                 4   34                       3       0   
...           ...               ...  ...                     ...     ...   
9607        34636                 1   43                       0       0   
9608        34962                 5   28                       2       0   
9609        39286                 3   31                       2       0   
9610        39145                 4   32                       4       0   
9611        90726                 1   46                       0       0   

      IS_FIRST_JOB  TRAVELLED_REQUIRED  PERFORMANCE_RATING  DISABLED_EMP  \
0                0                   0                   4             0   
1                1                   0                   3             0   
2                1                   0                   3             0   
3                0                   1                   2             0   
4                0                   0                   4             0   
...            ...                 ...                 ...           ...   
9607             0                   0                   1             0   
9608             0                   0                   1             0   
9609             0                   0                   1             0   
9610             0                   0                   4             0   
9611             0                   0                   1             0   

      DISABLED_VET  ...  MARITAL_STATUS_Married  MARITAL_STATUS_Single  \
0                0  ...                       0                      0   
1                0  ...                       0                      1   
2                0  ...                       0                      1   
3                0  ...                       0                      1   
4                0  ...                       0                      1   
...            ...  ...                     ...                    ...   
9607             1  ...                       0                      1   
9608             0  ...                       0                      1   
9609             0  ...                       0                      1   
9610             0  ...                       1                      0   
9611             0  ...                       0                      1   

      ETHNICITY_   ETHNICITY_AMIND  ETHNICITY_ASIAN  ETHNICITY_BLACK  \
0               0                0                0                1   
1               0                0                1                0   
2               0                0                0                0   
3               0                0                1                0   
4               0                0                1                0   
...           ...              ...              ...              ...   
9607            0                0                1                0   
9608            0                0                0                0   
9609            0                0                0                0   
9610            0                0                0                0   
9611            0                0                0                0   

      ETHNICITY_HISPA  ETHNICITY_PACIF  ETHNICITY_TWO  ETHNICITY_WHITE  
0                   0                0              0                0  
1                   0                0              0                0  
2                   0                0              0                1  
3                   0                0              0                0  
4                   0                0              0        

In [14]:
X = df.drop(['STATUS'], axis=1)
y = df['STATUS']

In [15]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [16]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
    
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=25, batch_size=16, validation_split = .2)

Epoch 1/25
337/337 [==============================] - 1s 3ms/step - loss: 0.0979 - accuracy: 0.9647 - val_loss: 2.0217 - val_accuracy: 0.6211
Epoch 2/25
337/337 [==============================] - 1s 2ms/step - loss: 0.0953 - accuracy: 0.9662 - val_loss: 2.0297 - val_accuracy: 0.6048
Epoch 3/25
337/337 [==============================] - 1s 2ms/step - loss: 0.0918 - accuracy: 0.9660 - val_loss: 2.4512 - val_accuracy: 0.6114
Epoch 4/25
337/337 [==============================] - 1s 2ms/step - loss: 0.0902 - accuracy: 0.9686 - val_loss: 2.1255 - val_accuracy: 0.6092
Epoch 5/25
337/337 [==============================] - 1s 2ms/step - loss: 0.0669 - accuracy: 0.9799 - val_loss: 2.2152 - val_accuracy: 0.6040
Epoch 6/25
337/337 [==============================] - 1s 2ms/step - loss: 0.0787 - accuracy: 0.9745 - val_loss: 2.3946 - val_accuracy: 0.6152
Epoch 7/25
337/337 [==============================] - 1s 2ms/step - loss: 0.0566 - accuracy: 0.9823 - val_loss: 2.5883 - val_accuracy: 0.5958
Epoch 

In [20]:
y_pred = model.predict(X_test)
y_pred = [1 if i >= 0.5 else 0 for i in y_pred]

91/91 [==============================] - 0s 1ms/step


In [21]:
pd.DataFrame(confusion_matrix(y_test,y_pred), columns=["Predicted A", "Predicted T"], index=["Actual A","Actual T"] )

Predicted A  Predicted T
Actual A         1073          552
Actual T          550          709

In [22]:
score = accuracy_score(y_test, y_pred)
print('Model Accuracy:', score)
print('Classification Report:\n', classification_report(y_test, y_pred))

Model Accuracy: 0.6178918169209431
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.66      0.66      1625
           1       0.56      0.56      0.56      1259

    accuracy                           0.62      2884
   macro avg       0.61      0.61      0.61      2884
weighted avg       0.62      0.62      0.62      2884

